**Exercise 2: Linear Regression**

*CPSC 381/581: Machine Learning*

*Yale University*

*Instructor: Alex Wong*


**Prerequisites**:

1. Enable Google Colaboratory as an app on your Google Drive account

2. Create a new Google Colab notebook, this will also create a "Colab Notebooks" directory under "MyDrive" i.e.
```
/content/drive/MyDrive/Colab Notebooks
```

3. Create the following directory structure in your Google Drive
```
/content/drive/MyDrive/Colab Notebooks/CPSC 381-581: Machine Learning/Exercises
```

4. Move the 02_exercise_linear_regression.ipynb into
```
/content/drive/MyDrive/Colab Notebooks/CPSC 381-581: Machine Learning/Exercises
```
so that its absolute path is
```
/content/drive/MyDrive/Colab Notebooks/CPSC 381-581: Machine Learning/Exercises/01_exercise_linear_regression.ipynb
```

In this exercise, we will optimize a linear function for the regression task using the normal equation and pseudo-inverse methods. We will test them on several datasets.


**Submission**:

1. Implement all TODOs in the code blocks below.

2. Report your training, validation, and testing scores.

```
Report training, validation, and testing scores here.

Note: for full points, you must be able to reproduce the same scores as linear regression from scikit learn.
```

3. List any collaborators.

```
Collaborators: Doe, Jane (Please write names in <Last Name, First Name> format)

Collaboration details: Discussed ... implementation details with Jane Doe.
```

Import packages

In [1]:
import numpy as np
import sklearn.datasets as skdata
import sklearn.metrics as skmetrics
from sklearn.linear_model import LinearRegression as LinearRegressionSciKit
import warnings

warnings.filterwarnings(action='ignore')
np.random.seed = 1

Loading data

In [ ]:
'''
Implementation of linear regression by directly solving normal equation or pseudo-inverse
'''
class LinearRegression(object):

    def __init__(self):
        # Define private variables
        self.__weights = None

    def __fit_normal_equation(self, X, y):
        '''
        Fits the model to x and y via normal equation

        Arg(s):
            X : numpy
                N x d feature vector
            y : numpy
                N x 1 ground-truth label
        '''

        # DONE: Implement the __fit_normal_equation function
        # Normal equation: w* = (X,TX)^-1 X,Ty
        X_t_X = np.matmul(X.T, X)
        X_t_X_inverse = np.linalg.inv(X_t_X)

        self.__weights = np.matmul(np.matmul(X_t_X_inverse, X.T), y)

    def __fit_pseudoinverse(self, X, y):
        '''
        Fits the model to x and y via pseudo-inverse

        Arg(s):
            X : numpy
                N x d feature vector
            y : numpy
                N x 1 ground-truth label
        '''

        # DONE: Implement the __fit_pseudoinverse function
        # SVD: U,S,V_t
        U, S, V_t = np.linalg.svd(X)
        # Reciprocal of values w/ diagonal matrix
        S_plus = np.diag(1.0/S) # = d * d matrix (d, d)

        # Pad for multiplication purposes
        padding = np.zeros([U.shape[0], S.shape[0]])
        #S+
        S_pseudo = np.concat([S_plus, padding], axis=0)

        #X+ = V S+
        # Pseudo inverse of X
        X_pseudo = np.matmul([V_t.T, S_pseudo, U.T])

        # Solution
        self.__weights = np.matmul(X_pseudo, y)

    def fit(self, x, y, solver='normal_equation'):
        '''
        Fits the model to x and y by solving the ordinary least squares
        using normal equation or pseudoinverse (SVD)

        Arg(s):
            x : numpy[float32]
                d x N feature vector
            y : numpy[float32]
                1 x N ground-truth label
            solver : str
                solver types: normal_equation, pseudoinverse
        '''

        # DONE: Implement the fit function
        X = x.T
        Y = y.T

        if solver == 'normal_equation':
            self.__fit_normal_equation(X, Y)
        elif solver == 'pseudoinverse':
            self.__fit_pseudoinverse(X, Y)
        else:
            raise ValueError('Encountered unsupported solver: {}'.format(solver))

    def predict(self, x):
        '''
        Predicts the real value for each feature vector x

        Arg(s):
            x : numpy[float32]
                d x N feature vector
        Returns:
            numpy[float32] : d x 1 real value vector (\hat{y})
        '''

        # DONE: Implement the predict function
        return np.matmul(self.weights.T, x)

    def __score_r_squared(self, y_hat, y):
        '''
        Measures the r-squared score from groundtruth y

        Args:
            y_hat : numpy[float32]
                1 x N predictions
            y : numpy[float32]
                1 x N ground-truth label

        Returns:
            float : r-squared score
        '''

        # DONE: Implement the __score_r_squared function
        # Sum of square is unexplained variation
        # R2 = 1 - u/v
        #u = sum (y_hat - y)^2 (predictions - groud_truth) = sum sq errors
        #v = sum (y - y_mean)^2  = sum sq unexplained var
        
        u = np.sum((y_hat - y) ** 2)
        v = np.sum((y - np.mean(y)) ** 2)

        return 1 - (u/v)


    def __score_mean_squared_error(self, y_hat, y):
        '''
        Measures the mean squared error (distance) from groundtruth y

        Arg(s):
            y_hat : numpy[float32]
                1 x N predictions
            y : numpy[float32]
                1 x N ground-truth label

        Returns:
            float : mean squared error (mse)
        '''

        # DONE: Implement the __score_mean_squared_error function
        return np.mean((y_hat - y) ** 2)

    def score(self, x, y, scoring_func='r_squared'):
        '''
        Predicts real values from x and measures the mean squared error (distance)
        or r-squared from groundtruth y

        Arg(s):
            x : numpy[float32]
                d x N feature vector
            y : numpy[float32]
                1 x N ground-truth label
            scoring_func : str
                scoring function: r_squared, mean_squared_error

        Returns:
            float : mean squared error (mse)
        '''

        # DONE: Implement the score function
        y_hat = self.__predict(x)

        if scoring_func == 'r_squared':
            return self.__score_r_squared(y_hat, y) # gives R2 scores
        elif scoring_func == 'mean_squared_error':
            return self.__score_mean_squared_error(y_hat, y)
        else:
            raise ValueError('Encountered unsupported scoring function: {}'.format(scoring_func))


Implementing training and validation loop for linear regression

In [ ]:
# Load diabetes and California housing prices dataset
datasets = [skdata.load_diabetes(), skdata.fetch_california_housing()]
dataset_names = ['diabetes', 'California housing prices']

for dataset, dataset_name in zip(datasets, dataset_names):
    '''
    Create the training, validation and testing splits
    '''
    x = dataset.data
    y = dataset.target

    # Shuffle the dataset based on sample indices
    shuffled_indices = np.random.permutation(x.shape[0])

    # Choose the first 80% as training set, next 10% as validation and the rest as testing
    train_split_idx = int(0.80 * x.shape[0])
    val_split_idx = int(0.90 * x.shape[0])

    train_indices = shuffled_indices[0:train_split_idx]
    val_indices = shuffled_indices[train_split_idx:val_split_idx]
    test_indices = shuffled_indices[val_split_idx:]

    # Select the examples from x and y to construct our training, validation, testing sets
    x_train, y_train = x[train_indices, :], y[train_indices]
    x_val, y_val = x[val_indices, :], y[val_indices]
    x_test, y_test = x[test_indices, :], y[test_indices]

    '''
    Trains and tests Perceptron model from scikit-learn
    '''
    # DONE: Initialize scikit-learn linear regression model without bias
    model_scikit = LinearRegressionSciKit(fit_intercept=False) # no b

    # DONE: Trains scikit-learn linear regression model
    model_scikit.fit(x_train, y_train)

    print('***** Results of scikit-learn linear regression model on {} dataset *****'.format(
        dataset_name))

    # DONE: Test model on training set
    predictions_train = model_scikit.predict()

    score_mse_train = skmetrics.mean_squared_error(y_train, predictions_train)
    print('Training set mean squared error: {:.4f}'.format(score_mse_train))

    score_r2_train = skmetrics.r2_score(y_train, predictions_train)
    print('Training set r-squared scores: {:.4f}'.format(score_r2_train))

# repeat above for validation and testing
    # TODO: Test model on validation set
    predictions_val =  None

    score_mse_val = 0.0
    print('Validation set mean squared error: {:.4f}'.format(score_mse_val))

    score_r2_val = 0.0
    print('Validation set r-squared scores: {:.4f}'.format(score_r2_val))

    # TODO: Test model on testing set
    predictions_test = None

    score_mse_test = 0.0
    print('Testing set mean squared error: {:.4f}'.format(score_mse_test))

    score_r2_test = 0.0
    print('Testing set r-squared scores: {:.4f}'.format(score_r2_test))

    '''
    Trains and tests our linear regression model using different solvers
    '''

    # TODO: Take the transpose of the dataset to match the dimensions discussed in lecture
    # i.e., (N x d) to (d x N)

    x_train = None
    x_val = None
    x_test = None

    # Train 2 linear regression models using normal equation and pseudoinverse
    solvers = ['normal_equation', 'pseudoinverse']

    for solver in solvers:
        # TODO: Initialize our linear regression model
        model_ours = None

        print('***** Results of our linear regression model trained with {} on {} dataset *****'.format(
            solver, dataset_name))

        # TODO: Train model on training set

        # TODO: Test model on training set using mean squared error and r-squared
        score_mse_train = 0.0
        print('Training set mean squared error: {:.4f}'.format(score_mse_train))

        score_r2_train = 0.0
        print('Training set r-squared scores: {:.4f}'.format(score_r2_train))

        # TODO: Test model on validation set using mean squared error and r-squared
        score_mse_val = 0.0
        print('Validation set mean squared error: {:.4f}'.format(score_mse_val))

        score_r2_val = 0.0
        print('Validation set r-squared scores: {:.4f}'.format(score_r2_val))

        # TODO: Test model on testing set using mean squared error and r-squared
        score_mse_test = 0.0
        print('Testing set mean squared error: {:.4f}'.format(score_mse_test))

        score_r2_test = 0.0
        print('Testing set r-squared scores: {:.4f}'.format(score_r2_test))